# Cloud Workshop Azure Databricks
## 15. MLFlow avec PyTorch
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.3 12/06/2019

# MLflow PyTorch Notebook

This is an MLflow PyTorch notebook is based on [MLflow's PyTorch TensorBoard tutorial](https://github.com/mlflow/mlflow/blob/master/examples/pytorch/mnist_tensorboard_artifact.py).

- This notebook demonstrates how to run PyTorch to fit a neural network on MNIST handwritten digit recognition data.
- The run results are logged to an MLFlow server. 
- Training metrics and weights in TensorFlow event format are logged locally and then uploaded to the MLflow run's artifact directory.
- TensorBoard is started on the local log and then optionally on the uploaded log.

In this tutorial you:

- Create a **GPU-enabled cluster**
- Install the MLflow library on the cluster
- Run a neural network on MNIST handwritten digit recognition data
- View the results of training the network in the MLflow experiment UI
- View the results of training the network in TensorBoard

### Create a cluster and install MLflow on your cluster

1. Create a GPU-enabled cluster specifying:
    - **Databricks Runtime Version:** Databricks Runtime 5.0 ML Beta **GPU** or above
    - **Python Version:** Python 3
1. Install required library.
   1. Create required library.
    * Source **PyPI** and enter `mlflow`.
   1. Install the library into the cluster.
1. Attach this notebook to the cluster.

In [4]:
import sys
sys.version

## 1. Train an MNIST digit recognizer using PyTorch

#### Organize MLflow Runs into Experiments

In [7]:
# Trains using PyTorch and logs training metrics and weights in TensorFlow event format to the MLflow run's artifact directory. 
# This stores the TensorFlow events in MLflow for later access using TensorBoard.
#
# Code based on https://github.com/mlflow/mlflow/blob/master/example/tutorial/pytorch_tensorboard.py.


from __future__ import print_function
import os
import tempfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from collections import namedtuple
import tensorflow as tf
import tensorflow.summary
from tensorflow.summary import scalar
from tensorflow.summary import histogram
from chardet.universaldetector import UniversalDetector

# Create Params dictionary
class Params(object):
	def __init__(self, batch_size, test_batch_size, epochs, lr, momentum, seed, cuda, log_interval):
		self.batch_size = batch_size
		self.test_batch_size = test_batch_size
		self.epochs = epochs
		self.lr = lr
		self.momentum = momentum
		self.seed = seed
		self.cuda = cuda
		self.log_interval = log_interval

# Configure args
args = Params(64, 1000, 10, 0.01, 0.5, 1, True, 200)

cuda = not args.cuda and torch.cuda.is_available()


kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

    def log_weights(self, step):
        writer.add_summary(histogram('weights/conv1/weight', model.conv1.weight.data).eval(), step)
        writer.add_summary(histogram('weights/conv1/bias', model.conv1.bias.data).eval(), step)
        writer.add_summary(histogram('weights/conv2/weight', model.conv2.weight.data).eval(), step)
        writer.add_summary(histogram('weights/conv2/bias', model.conv2.bias.data).eval(), step)
        writer.add_summary(histogram('weights/fc1/weight', model.fc1.weight.data).eval(), step)
        writer.add_summary(histogram('weights/fc1/bias', model.fc1.bias.data).eval(), step)
        writer.add_summary(histogram('weights/fc2/weight', model.fc2.weight.data).eval(), step)
        writer.add_summary(histogram('weights/fc2/bias', model.fc2.bias.data).eval(), step)

model = Model()
if cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

writer = None # Will be used to write TensorBoard events

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            step = epoch * len(train_loader) + batch_idx
            log_scalar('train_loss', loss.data.item(), step)
            model.log_weights(step)

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').data.item() # sum up batch loss
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))
    step = (epoch + 1) * len(train_loader)
    log_scalar('test_loss', test_loss, step)
    log_scalar('test_accuracy', test_accuracy, step)

def log_scalar(name, value, step):
    """Log a scalar value to both MLflow and TensorBoard"""
    writer.add_summary(scalar(name, value).eval(), step)
    mlflow.log_metric(name, value)

## 2. Create a TensorFlow session and start MLflow

In [9]:
import mlflow.pytorch

sess = tf.InteractiveSession()
with mlflow.start_run() as run:  
  # Log our parameters into mlflow
  for key, value in vars(args).items():
      mlflow.log_param(key, value)

  output_dir = tempfile.mkdtemp()
  print("Writing TensorFlow events locally to %s\n" % output_dir)
  writer = tf.summary.FileWriter(output_dir, graph=sess.graph) 

  for epoch in range(1, args.epochs + 1):
      # print out active_run
      print("Active Run ID: %s, Epoch: %s \n" % (run.info.run_uuid, epoch))

      train(epoch)
      test(epoch)
      
  print("Uploading TensorFlow events as a run artifact.")
  mlflow.log_artifacts(output_dir, artifact_path="events")

Writing TensorFlow events locally to /tmp/tmpabd7f0_1

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 1 

Train Epoch: 1 [0/60000 (0%)]	Loss: 4.197468
Train Epoch: 1 [12800/60000 (21%)]	Loss: 3.365239
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.860766
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.725742
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.662532

Test set: Average loss: 2.2091, Accuracy: 9339/10000 (93%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 2 

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.606720
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.556422
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.501592
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.630374
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.420966

Test set: Average loss: 2.0814, Accuracy: 9566/10000 (96%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 3 

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.290576
Train Epoch: 3 [12800/60000 (21%)]	Loss: 2.216802
Train Epoch: 3 [25600/60000 (43%)]	Loss: 2.456002
Train Epoch: 3 [38400/60000 (64%)]	Loss: 2.397808
Train Epoch: 3 [51200/60000 (85%)]	Loss: 2.426887

Test set: Average loss: 2.0483, Accuracy: 9665/10000 (97%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 4 

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.225744
Train Epoch: 4 [12800/60000 (21%)]	Loss: 2.489834
Train Epoch: 4 [25600/60000 (43%)]	Loss: 2.139439
Train Epoch: 4 [38400/60000 (64%)]	Loss: 2.164264
Train Epoch: 4 [51200/60000 (85%)]	Loss: 2.166989

Test set: Average loss: 2.0281, Accuracy: 9684/10000 (97%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 5 

Train Epoch: 5 [0/60000 (0%)]	Loss: 2.098589
Train Epoch: 5 [12800/60000 (21%)]	Loss: 2.046174
Train Epoch: 5 [25600/60000 (43%)]	Loss: 2.181350
Train Epoch: 5 [38400/60000 (64%)]	Loss: 2.194591
Train Epoch: 5 [51200/60000 (85%)]	Loss: 2.279545

Test set: Average loss: 2.0219, Accuracy: 9703/10000 (97%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 6 

Train Epoch: 6 [0/60000 (0%)]	Loss: 2.260291
Train Epoch: 6 [12800/60000 (21%)]	Loss: 2.120440
Train Epoch: 6 [25600/60000 (43%)]	Loss: 2.242338
Train Epoch: 6 [38400/60000 (64%)]	Loss: 2.151783
Train Epoch: 6 [51200/60000 (85%)]	Loss: 2.193433

Test set: Average loss: 2.0111, Accuracy: 9719/10000 (97%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 7 

Train Epoch: 7 [0/60000 (0%)]	Loss: 2.162578
Train Epoch: 7 [12800/60000 (21%)]	Loss: 2.197510
Train Epoch: 7 [25600/60000 (43%)]	Loss: 2.198035
Train Epoch: 7 [38400/60000 (64%)]	Loss: 2.074801
Train Epoch: 7 [51200/60000 (85%)]	Loss: 2.224810

Test set: Average loss: 2.0071, Accuracy: 9773/10000 (98%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 8 

Train Epoch: 8 [0/60000 (0%)]	Loss: 2.086279
Train Epoch: 8 [12800/60000 (21%)]	Loss: 2.084762
Train Epoch: 8 [25600/60000 (43%)]	Loss: 2.158970
Train Epoch: 8 [38400/60000 (64%)]	Loss: 2.214148
Train Epoch: 8 [51200/60000 (85%)]	Loss: 2.097444

Test set: Average loss: 2.0007, Accuracy: 9785/10000 (98%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 9 

Train Epoch: 9 [0/60000 (0%)]	Loss: 2.211730
Train Epoch: 9 [12800/60000 (21%)]	Loss: 2.040725
Train Epoch: 9 [25600/60000 (43%)]	Loss: 2.116132
Train Epoch: 9 [38400/60000 (64%)]	Loss: 2.141839
Train Epoch: 9 [51200/60000 (85%)]	Loss: 2.048464

Test set: Average loss: 2.0005, Accuracy: 9773/10000 (98%)

Active Run ID: 9ac13f5dedf84afb86b3371959e77fdf, Epoch: 10 

Train Epoch: 10 [0/60000 (0%)]	Loss: 2.145846
Train Epoch: 10 [12800/60000 (21%)]	Loss: 2.121406
Train Epoch: 10 [25600/60000 (43%)]	Loss: 2.185719
Train Epoch: 10 [38400/60000 (64%)]	Loss: 2.078063
Train Epoch: 10 [51200/60000 (85%)]	Loss: 2.064973

Test set: Average loss: 1.9932, Accuracy: 9785/10000 (98%)

Uploading TensorFlow events as a run artifact.

## 3. Review the experiment using MLFlow integration with Azure Databricks

> Cliquer sur le bouton **Runs** en haut à droite pour voir les runs MLFlow

#### MLflow UI for the PyTorch MNIST Run
<img src="https://docs.databricks.com/_static/images/mlflow/mlflow-pytorch-mlflow-ui.gif" width=1000/>

## 4. Start TensorBoard on local directory

In [14]:
dbutils.tensorboard.start(output_dir)

TensorBoard log directory set to: /tmp/tmpabd7f0_1. 
 View TensorBoard

### View the results in TensorBoard

The computations you'll use TensorFlow for - like training a massive deep neural network - can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, we've included a suite of visualization tools called TensorBoard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it.

https://www.tensorflow.org/guide/summaries_and_tensorboard


<img src="https://docs.databricks.com/_static/images/third-party-integrations/tensorflow/tensorboard.png"/>

Click the **View TensorBoard** link. It should look like the following:

#### TensorBoard for the PyTorch MNIST Run
<img src="https://docs.databricks.com/_static/images/mlflow/mlflow-pytorch-tensorboard.gif" width=1000/>

## Stop TensorBoard

In [18]:
dbutils.tensorboard.stop()

Looking for active tensorboard process...
Active tensorboard process killed...
Out[5]: True

> Fin